## Day5-01: Agent2Agent (A2A) 通信协议

5 天 AI Agents 强化课程的第五天第一部分，我们将学习如何构建**多 Agent 系统**，让不同的 Agent 能够使用 **Agent2Agent (A2A) 协议**进行通信和协作。

你将学会：

- 理解 A2A 协议以及何时使用它 vs 子 Agent
- A2A 的常见架构模式（跨框架、跨语言、跨组织）
- 使用 `to_a2a()` 将 ADK Agent 暴露为 A2A 服务
- 使用 `RemoteA2aAgent` 消费远程 Agent
- 构建一个产品目录集成系统

## 精简总结：A2A 协议

### 核心问题

- **单个 Agent 无法完成所有任务** - 专门化的 Agent 处理不同领域效果更好
- **Agent 需要协作** - 客服需要产品数据，订单系统需要库存信息
- **不同团队构建不同 Agent** - 你需要集成外部供应商的 Agent
- **Agent 可能使用不同语言/框架** - 需要标准通信协议

### A2A vs 本地子 Agent 决策表

| 因素 | 使用 A2A | 使用本地子 Agent |
|------|---------|------------------|
| Agent 位置 | 外部服务，不同代码库 | 同一代码库，内部 |
| 所有权 | 不同团队/组织 | 你的团队 |
| 网络 | 不同机器上的 Agent | 同一进程/机器 |
| 性能 | 可接受网络延迟 | 需要低延迟 |
| 语言/框架 | 需要跨语言/框架 | 相同语言 |
| 合约 | 需要正式 API 合约 | 内部接口 |

### 关键组件

| 组件 | 作用 |
|------|------|
| `to_a2a()` | 将 Agent 暴露为 A2A 兼容服务 |
| Agent Card | Agent 的"名片"，描述能力和端点 |
| `RemoteA2aAgent` | 连接远程 Agent 的客户端代理 |

### 本教程构建的系统

```text
┌──────────────────────┐           ┌──────────────────────┐
│ 客服 Agent           │  ─A2A──▶  │ 产品目录 Agent        │
│ (消费者)             │           │ (外部供应商)          │
│ localhost:8000       │           │ localhost:8001        │
└──────────────────────┘           └──────────────────────┘
```

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## 📚 A2A 协议概述

### A2A 协议是什么？

[Agent2Agent (A2A) 协议](https://a2a-protocol.org/) 是一个**标准**，允许 Agent：
- ✨ **通过网络通信** - Agent 可以在不同机器上
- ✨ **使用彼此的能力** - 一个 Agent 可以像工具一样调用另一个 Agent
- ✨ **跨框架工作** - 语言/框架无关
- ✨ **维护正式合约** - Agent Card 描述能力

### 🏗️ 常见 A2A 架构模式

A2A 协议在三种场景中特别有用：

1. **跨框架集成**：ADK Agent 与其他 Agent 框架通信
2. **跨语言通信**：Python Agent 调用 Java 或 Node.js Agent
3. **跨组织边界**：你的内部 Agent 与外部供应商服务集成

![何时选择 A2A？](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_01.png)

### 🔄 我们将构建的系统

![系统架构](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_02.png)

**工作流程：**
1. **客户** 向客服 Agent 询问产品问题
2. **客服 Agent** 意识到需要产品信息
3. **客服 Agent** 通过 A2A 协议调用 **产品目录 Agent**
4. **产品目录 Agent**（外部供应商）返回产品数据
5. **客服 Agent** 组织答案并响应客户

---
## ⚙️ Section 1: 设置环境

### 1.1 配置你的 Gemini API 密钥

In [ ]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key 配置完成。")
            break
else:
    print(f"请在以下路径创建 .env 文件: {env_file}")

### 1.2 导入 ADK 组件

In [ ]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# 隐藏额外的警告
import warnings
warnings.filterwarnings("ignore")

print("✅ ADK 组件导入成功。")

### 1.3 配置重试选项

使用 LLM 时，你可能会遇到临时错误，如速率限制或服务暂时不可用。重试选项通过指数退避自动处理这些失败。

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)

---
## 📦 Section 2: 创建产品目录 Agent（将被暴露）

我们将创建一个**产品目录 Agent**，从外部供应商的目录中提供产品信息。这个 Agent 将**通过 A2A 暴露**，以便其他 Agent（如客服）可以使用它。

### 为什么要暴露这个 Agent？
- 在真实系统中，这将由**外部供应商**或第三方提供商维护
- 你的内部 Agent（客服、销售、库存）需要产品数据
- 供应商**控制自己的代码库** - 你无法修改他们的实现
- 通过 A2A 暴露它，任何授权的 Agent 都可以使用标准协议消费它

In [ ]:
# 定义产品目录查询工具
# 在真实系统中，这会查询供应商的产品数据库
def get_product_info(product_name: str) -> str:
    """获取指定产品的信息。

    Args:
        product_name: 产品名称（例如 "iPhone 15 Pro"、"MacBook Pro"）

    Returns:
        产品信息字符串
    """
    # 模拟产品目录 - 生产环境中会查询真实数据库
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, 库存紧张 (8台), 128GB, 钛金属外壳",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, 有货 (31台), 256GB, 幻影黑",
        "dell xps 15": 'Dell XPS 15, $1,299, 有货 (45台), 15.6" 显示屏, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14", $1,999, 有货 (22台), M3 Pro 芯片, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 耳机, $399, 有货 (67台), 降噪, 30小时续航",
        "ipad air": 'iPad Air, $599, 有货 (28台), 10.9" 显示屏, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34" 显示器, $499, 缺货, 预计: 下周到货',
    }

    product_lower = product_name.lower().strip()

    if product_lower in product_catalog:
        return f"产品: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"抱歉，我没有 {product_name} 的信息。可用产品: {available}"


# 创建产品目录 Agent
# 这个 Agent 专门从供应商目录提供产品信息
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="外部供应商的产品目录 Agent，提供产品信息和库存状态。",
    instruction="""
    你是来自外部供应商的产品目录专家。
    当被问到产品时，使用 get_product_info 工具从目录中获取数据。
    提供清晰、准确的产品信息，包括价格、库存和规格。
    如果被问到多个产品，逐一查询。
    保持专业和友好。
    """,
    tools=[get_product_info],  # 注册产品查询工具
)

print("✅ 产品目录 Agent 创建成功！")
print("   模型: gemini-2.5-flash-lite")
print("   工具: get_product_info()")
print("   准备通过 A2A 暴露...")

---
## 🌐 Section 3: 通过 A2A 暴露产品目录 Agent

现在我们将使用 ADK 的 `to_a2a()` 函数使产品目录 Agent 可被其他 Agent 访问。

### `to_a2a()` 做了什么：
- 🔧 将你的 Agent 包装在 A2A 兼容的服务器中（FastAPI/Starlette）
- 📋 自动生成 **Agent Card**，包含：
  - Agent 名称、描述和版本
  - 技能（你的工具/函数在 A2A 中成为"技能"）
  - 协议版本和端点
  - 输入/输出模式
- 🌐 在 `/.well-known/agent-card.json` 提供 Agent Card（标准 A2A 路径）
- ✨ 处理所有 A2A 协议细节

### 💡 关键概念：Agent Card

**Agent Card** 是一个 JSON 文档，作为你 Agent 的"名片"。它描述：
- Agent 做什么（名称、描述、版本）
- 有什么能力（技能、工具、函数）
- 如何与它通信（URL、协议版本、端点）

每个 A2A Agent 必须在标准路径发布其 Agent Card：`/.well-known/agent-card.json`

In [ ]:
# 将产品目录 Agent 转换为 A2A 兼容的应用
# 这创建了一个 FastAPI/Starlette 应用，它：
#   1. 在 A2A 协议端点提供 Agent 服务
#   2. 提供自动生成的 Agent Card
#   3. 处理 A2A 通信协议
product_catalog_a2a_app = to_a2a(
    product_catalog_agent, port=8001  # Agent 将在此端口提供服务
)

print("✅ 产品目录 Agent 现在是 A2A 兼容的！")
print("   Agent 将在: http://localhost:8001 提供服务")
print("   Agent Card 在: http://localhost:8001/.well-known/agent-card.json")
print("   准备启动服务器...")

---
## 🚀 Section 4: 启动产品目录 Agent 服务器

我们将使用 `uvicorn` 在**后台**启动产品目录 Agent 服务器，以便它可以处理来自其他 Agent 的请求。

### 为什么在后台运行？
- 服务器需要在我们创建和测试客服 Agent 时持续运行
- 这模拟了真实场景，不同 Agent 作为独立服务运行
- 在生产环境中，供应商会在他们的基础设施上托管这个服务

In [ ]:
# 首先，将产品目录 Agent 保存到 uvicorn 可以导入的文件
product_catalog_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

def get_product_info(product_name: str) -> str:
    """获取指定产品的信息。"""
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, 库存紧张 (8台), 128GB, 钛金属外壳",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, 有货 (31台), 256GB, 幻影黑",
        "dell xps 15": "Dell XPS 15, $1,299, 有货 (45台), 15.6\\" 显示屏, 16GB RAM, 512GB SSD",
        "macbook pro 14": "MacBook Pro 14\\", $1,999, 有货 (22台), M3 Pro 芯片, 18GB RAM, 512GB SSD",
        "sony wh-1000xm5": "Sony WH-1000XM5 耳机, $399, 有货 (67台), 降噪, 30小时续航",
        "ipad air": "iPad Air, $599, 有货 (28台), 10.9\\" 显示屏, 64GB",
        "lg ultrawide 34": "LG UltraWide 34\\" 显示器, $499, 缺货, 预计: 下周到货",
    }
    
    product_lower = product_name.lower().strip()
    
    if product_lower in product_catalog:
        return f"产品: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"抱歉，我没有 {product_name} 的信息。可用产品: {available}"

product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="外部供应商的产品目录 Agent，提供产品信息和库存状态。",
    instruction="""
    你是来自外部供应商的产品目录专家。
    当被问到产品时，使用 get_product_info 工具从目录中获取数据。
    提供清晰、准确的产品信息，包括价格、库存和规格。
    如果被问到多个产品，逐一查询。
    保持专业和友好。
    """,
    tools=[get_product_info]
)

# 创建 A2A 应用
app = to_a2a(product_catalog_agent, port=8001)
'''

# 将产品目录 Agent 写入临时文件
with open("/tmp/product_catalog_server.py", "w") as f:
    f.write(product_catalog_agent_code)

print("📝 产品目录 Agent 代码已保存到 /tmp/product_catalog_server.py")

# 在后台启动 uvicorn 服务器
server_process = subprocess.Popen(
    [
        "uvicorn",
        "product_catalog_server:app",
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)

print("🚀 正在启动产品目录 Agent 服务器...")
print("   等待服务器就绪...")

# 等待服务器启动（轮询直到响应）
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ 产品目录 Agent 服务器正在运行！")
            print(f"   服务器 URL: http://localhost:8001")
            print(f"   Agent Card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️ 服务器可能还未就绪。如需要请手动检查。")

# 存储进程以便稍后停止
globals()["product_catalog_server_process"] = server_process

### 🔍 查看自动生成的 Agent Card

`to_a2a()` 函数自动创建了一个描述产品目录 Agent 能力的 **Agent Card**。让我们看看！

In [ ]:
# 从运行中的服务器获取 Agent Card
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 产品目录 Agent Card:")
        print(json.dumps(agent_card, indent=2, ensure_ascii=False))

        print("\n✨ 关键信息:")
        print(f"   名称: {agent_card.get('name')}")
        print(f"   描述: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   技能: {len(agent_card.get('skills', []))} 个能力已暴露")
    else:
        print(f"❌ 获取 Agent Card 失败: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ 获取 Agent Card 出错: {e}")
    print("   确保产品目录 Agent 服务器正在运行（前一个 cell）")

---
## 🎧 Section 5: 创建客服 Agent（消费者）

现在我们将创建一个**客服 Agent**，通过 A2A 消费产品目录 Agent。

### 工作原理：
1. 我们使用 `RemoteA2aAgent` 创建产品目录 Agent 的**客户端代理**
2. 客服 Agent 可以像使用任何其他工具一样使用产品目录 Agent
3. ADK 在幕后处理所有 A2A 协议通信

这展示了 A2A 的强大：**Agent 可以像本地一样协作！**

### `RemoteA2aAgent` 工作原理：
- 它是一个**客户端代理**，读取远程 Agent 的 Card
- 将子 Agent 调用转换为 A2A 协议请求（HTTP POST 到 `/tasks`）
- 处理所有协议细节，你只需像使用普通子 Agent 一样使用它

In [ ]:
# 创建连接到产品目录 Agent 的 RemoteA2aAgent
# 这作为客户端代理 - 客服 Agent 可以像本地 Agent 一样使用它
remote_product_catalog_agent = RemoteA2aAgent(
    name="product_catalog_agent",
    description="来自外部供应商的远程产品目录 Agent，提供产品信息。",
    # 指向 Agent Card URL - 这是 A2A 协议元数据所在的位置
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ 远程产品目录 Agent 代理创建成功！")
print(f"   连接到: http://localhost:8001")
print(f"   Agent Card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   客服 Agent 现在可以像本地子 Agent 一样使用它！")

In [ ]:
# 现在创建使用远程产品目录 Agent 的客服 Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="customer_support_agent",
    description="帮助客户查询产品信息的客服助手。",
    instruction="""
    你是一个友好、专业的客服代表。
    
    当客户询问产品时：
    1. 使用 product_catalog_agent 子 Agent 查询产品信息
    2. 提供关于价格、库存和规格的清晰答案
    3. 如果产品缺货，提及预计到货时间
    4. 保持友好和专业！
    
    在回答客户问题前，始终从 product_catalog_agent 获取产品信息。
    """,
    sub_agents=[remote_product_catalog_agent],  # 将远程 Agent 添加为子 Agent！
)

print("✅ 客服 Agent 创建成功！")
print("   模型: gemini-2.5-flash-lite")
print("   子 Agent: 1 个（通过 A2A 连接的远程产品目录 Agent）")
print("   准备帮助客户！")

---
## 🧪 Section 6: 测试 A2A 通信

让我们测试 Agent 之间的通信！我们将向客服 Agent 询问产品，它将通过 A2A 与产品目录 Agent 通信。

### 幕后发生了什么：
1. 客户向客服 Agent 询问产品问题
2. 客服 Agent 意识到需要产品信息
3. 客服 Agent 调用 `remote_product_catalog_agent`（RemoteA2aAgent）
4. ADK 发送 A2A 协议请求到 `http://localhost:8001`
5. 产品目录 Agent 处理请求并响应
6. 客服 Agent 收到响应并继续
7. 客户得到最终答案

所有这些都是**透明**发生的 - 客服 Agent 不需要知道它在与远程 Agent 通信！

In [ ]:
async def test_a2a_communication(user_query: str):
    """
    测试客服 Agent 和产品目录 Agent 之间的 A2A 通信。

    此函数：
    1. 为这个对话创建新会话
    2. 将查询发送给客服 Agent
    3. 客服 Agent 通过 A2A 与产品目录 Agent 通信
    4. 显示响应

    Args:
        user_query: 向客服 Agent 提出的问题
    """
    # 设置会话管理（ADK 需要）
    session_service = InMemorySessionService()

    # 会话标识符
    app_name = "support_app"
    user_id = "demo_user"
    # 每次测试使用唯一的会话 ID 以避免冲突
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    # 关键：在运行 Agent 之前创建会话
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # 为客服 Agent 创建 runner
    runner = Runner(
        agent=customer_support_agent, app_name=app_name, session_service=session_service
    )

    # 创建用户消息
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # 显示查询
    print(f"\n👤 客户: {user_query}")
    print(f"\n🎧 客服 Agent 响应:")
    print("-" * 60)

    # 异步运行 Agent（处理流式响应和 A2A 通信）
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # 只打印最终响应（跳过中间事件）
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)


# 运行测试
print("🧪 测试 A2A 通信...\n")
await test_a2a_communication("请告诉我关于 iPhone 15 Pro 的信息？有货吗？")

### 尝试更多示例

让我们测试更多场景来看 A2A 通信的实际效果！

In [ ]:
# 测试比较多个产品
await test_a2a_communication(
    "我想买一台笔记本电脑。能帮我比较一下 Dell XPS 15 和 MacBook Pro 14 吗？"
)

In [ ]:
# 测试特定产品查询
await test_a2a_communication(
    "你们有 Sony WH-1000XM5 耳机吗？价格是多少？"
)

---
## 🔍 Section 7: 理解刚才发生了什么

### A2A 通信流程

当你运行上面的测试时，这是 Agent 如何通信的详细步骤：

![A2A 流程](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day5/a2a_03.png)

**发生了什么：**
1. **客户** 询问 iPhone 15 Pro
2. **客服 Agent**（LlmAgent）收到问题并决定需要产品信息
3. **客服 Agent** 委托给 `product_catalog_agent` 子 Agent
4. **RemoteA2aAgent**（客户端代理）将其转换为 A2A 协议请求
5. A2A 请求通过 HTTP 发送到 `http://localhost:8001`
6. **产品目录 Agent**（服务器）收到请求并调用 `get_product_info("iPhone 15 Pro")`
7. **产品目录 Agent** 通过 A2A 响应返回产品信息
8. **RemoteA2aAgent** 收到响应并传回给客服 Agent
9. **客服 Agent** 用产品详情组织最终答案
10. **客户** 收到完整、有帮助的响应

### 关键优势展示

1. **透明性**：客服 Agent 不"知道"产品目录 Agent 是远程的
2. **标准协议**：使用 A2A 标准 - 任何 A2A 兼容的 Agent 都能工作
3. **易于集成**：只需一行：`sub_agents=[remote_product_catalog_agent]`
4. **关注点分离**：产品数据在目录 Agent（供应商），客服逻辑在客服 Agent（你的公司）

### 真实世界应用

这种模式支持：
- **微服务**：每个 Agent 是独立服务
- **第三方集成**：消费外部供应商的 Agent（如产品目录、支付处理器）
- **跨语言**：产品目录 Agent 可以是 Java，客服 Agent 是 Python
- **专业团队**：供应商维护目录，你的团队维护客服 Agent
- **跨组织**：供应商在他们的基础设施托管目录，你通过 A2A 集成

---
## 🎓 下一步和学习资源

### 🚀 扩展想法

现在你理解了 A2A 基础，尝试扩展这个示例：

1. **添加更多 Agent**：
   - 创建一个检查库存水平和补货计划的 **库存 Agent**
   - 创建一个提供配送估计和跟踪的 **物流 Agent**
   - 让客服 Agent 通过 A2A 协调所有三个

2. **真实数据源**：
   - 用真实数据库替换模拟产品目录（PostgreSQL、MongoDB）
   - 添加真实库存跟踪系统集成
   - 连接真实支付网关 API

3. **高级 A2A 功能**：
   - 在 Agent 之间实现认证（API 密钥、OAuth）
   - 为网络故障添加错误处理和重试
   - 使用替代的 `adk api_server --a2a` 方法

### 📚 文档

**A2A 协议**：
- [官方 A2A 协议网站](https://a2a-protocol.org/)
- [A2A 协议规范](https://a2a-protocol.org/latest/spec/)

**ADK A2A 指南**：
- [ADK 中的 A2A 简介](https://google.github.io/adk-docs/a2a/intro/)
- [暴露 Agent 快速入门](https://google.github.io/adk-docs/a2a/quickstart-exposing/)
- [消费 Agent 快速入门](https://google.github.io/adk-docs/a2a/quickstart-consuming/)

**其他部署选项**：
- [部署 ADK Agent 到 Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/)
- [部署到 Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)
- [部署到 GKE](https://google.github.io/adk-docs/deploy/gke/)

---
## 📊 总结 - A2A 通信模式

### 关键要点

在这个 notebook 中，你学会了如何用 A2A 构建多 Agent 系统：

- **A2A 协议**：用于跨网络和框架的 Agent 间通信的标准化协议
- **暴露 Agent**：使用 `to_a2a()` 让你的 Agent 可被其他人访问，自动生成 Agent Card
- **消费 Agent**：使用 `RemoteA2aAgent` 将远程 Agent 集成为本地子 Agent
- **用例**：最适合微服务架构、跨团队集成和第三方 Agent 消费

---
## ✅ 恭喜！你是 A2A 专家了

你已经成功学会了如何使用 A2A 协议构建多 Agent 系统！

你现在知道如何将 Agent 暴露为服务、消费远程 Agent，以及构建可以跨团队和组织扩展的协作多 Agent 系统。

### 🎯 下一步

现在你理解了 A2A 通信，你可以构建复杂的多 Agent 系统，让专门的 Agent 协作解决真实世界的问题。考虑使用 Cloud Run 或 Agent Engine 将你的 Agent 部署到生产环境，使它们可以通过互联网访问！

准备好了吗？继续下一个 notebook 学习如何**将 Agent 部署到生产环境**！